In [73]:
import os
import sys
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

parent_dir = os.pardir
sys.path.insert(0, parent_dir)

from sklearn.datasets import load_iris
iris_data = load_iris()
X, y = iris_data.data, iris_data.target

from sklearn.model_selection import train_test_split

# Split the dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Print the shapes to confirm the sizes
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")
# convert to torch tensors
import torch
X_train = torch.from_numpy(X_train).float()
y_train = torch.squeeze(torch.from_numpy(y_train).long())
X_test = torch.from_numpy(X_test).float()
y_test = torch.squeeze(torch.from_numpy(y_test).long())

import numpy as np
# goup the train data in a tensor of the shape (n_classes, n_samples, n_features)
n_of_classes = len(torch.unique(y_train))
n_of_sets = 2
X_train_grouped = torch.stack([X_train[y_train == i] for i in range(n_of_classes)])
X_test_grouped = torch.stack([X_test[y_test == i] for i in range(n_of_classes)])
X_test_grouped = X_test_grouped.reshape(3, 2, -1, 4)
print(f"X_train_grouped shape: {X_train_grouped.shape}")
print(f"X_test_grouped shape: {X_test_grouped.shape}")

# convert tensors to numpy arrays
train_X = X_train_grouped.numpy()
train_y = np.arange(len(train_X))
test_X = X_test_grouped.numpy()
test_X = test_X.reshape(-1, test_X.shape[-2], test_X.shape[-1])
test_y = np.array([[i] * n_of_sets for i in range(n_of_classes)]).flatten()

print("Training data shape: {}".format(train_X.shape))
print("Training labels shape: {}".format(train_y.shape))
print("Test data shape: {}".format(test_X.shape))
print("Test labels shape: {}".format(test_y.shape))

from src.models import KernelMSM, MutualSubspaceMethod
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
# model = MutualSubspaceMethod(normalize=True, n_subdims=4, faster_mode=True)
model = KernelMSM(n_subdims=5, sigma=0.1, normalize=True, faster_mode=True)
model.fit(train_X, train_y)
pred = model.predict(test_X)
print(f"pred: {pred}\ntrue: {test_y}\naccuracy: {(pred == test_y).mean()}")

X_train shape: (120, 4)
y_train shape: (120,)
X_test shape: (30, 4)
y_test shape: (30,)
X_train_grouped shape: torch.Size([3, 40, 4])
X_test_grouped shape: torch.Size([3, 2, 5, 4])
Training data shape: (3, 40, 4)
Training labels shape: (3,)
Test data shape: (6, 5, 4)
Test labels shape: (6,)


100%|██████████| 3/3 [00:00<00:00, 4166.53it/s]

pred: [0 0 1 1 2 2]
true: [0 0 1 1 2 2]
accuracy: 1.0
